# 新版資料庫

搭啷！這個就是我們新版的資料庫，新增了
* 殖利率
* 三大法人
* 上櫃所有股票
* 更快速的存取方式

In [3]:
from finlab.crawler import (

    # 爬蟲們
    crawl_price,
    crawl_bargin,
    crawl_pe,
    crawl_monthly_report,
    crawl_finance_statement_by_date,
    crawl_twse_divide_ratio,
    crawl_otc_divide_ratio,
    crawl_twse_cap_reduction,
    crawl_otc_cap_reduction,
    crawl_benchmark,
    
    # 時間的range
    date_range,
    month_range,
    season_range,
    
    # GUI介面
    widget, out,
)

widget('price', crawl_price, date_range)
widget('bargin_report', crawl_bargin, date_range)
widget('pe', crawl_pe, date_range)
widget('monthly_report', crawl_monthly_report, month_range)
widget('benchmark', crawl_benchmark, date_range)
widget('financial_statement', crawl_finance_statement_by_date, season_range)
widget('twse_divide_ratio', crawl_twse_divide_ratio)
widget('otc_divide_ratio', crawl_otc_divide_ratio)
widget('twse_cap_reduction', crawl_twse_cap_reduction)
widget('otc_cap_reduction', crawl_otc_cap_reduction)

out.clear_output()
display(out)

AttributeError: Can't get attribute '_unpickle_timestamp' on <module 'pandas._libs.tslibs.timestamps' from '/home/fred/miniconda3/envs/finlab/lib/python3.7/site-packages/pandas/_libs/tslibs/timestamps.cpython-37m-x86_64-linux-gnu.so'>

In [4]:
from finlab.crawler import commit
commit()

已經成功commit過 balance_sheet 了，跳過！
已經成功commit過 bargin_report 了，跳過！
已經成功commit過 benchmark 了，跳過！
已經成功commit過 cash_flows 了，跳過！
已經成功commit過 forign_hold_ratio 了，跳過！
已經成功commit過 income_sheet 了，跳過！
已經成功commit過 income_sheet_cumulate 了，跳過！
已經成功commit過 monthly_report 了，跳過！
已經成功commit過 otc_cap_reduction 了，跳過！
已經成功commit過 otc_divide_ratio 了，跳過！
已經成功commit過 pe 了，跳過！
已經成功commit過 price 了，跳過！
已經成功commit過 twse_cap_reduction 了，跳過！
已經成功commit過 twse_divide_ratio 了，跳過！


In [ ]:
from finlab.data import Data

data = Data()

close = data.get('收盤價')
close.tail()

## 1. 資料調用

In [2]:
import pandas as pd
df = pd.read_pickle("history/tables/price.pkl") #history/tables/price.pkl
df.tail(24)

AttributeError: Can't get attribute 'new_block' on <module 'pandas.core.internals.blocks' from '/home/fred/miniconda3/envs/finlab/lib/python3.7/site-packages/pandas/core/internals/blocks.py'>

## 2. 分解上述的功能

### 2.A 分解其中的功能 crawl_function

In [ ]:
import datetime
from finlab.crawler import crawl_price
df = crawl_price(datetime.date(2019,1,2))
df.tail()

### 2.B 分解其中功能 time_range

In [ ]:
season_range(datetime.date(2018,1,2), datetime.date(2019,1,10))

### 2.C 將功能組裝起來 widget

In [ ]:
widget("price", crawl_price, date_range)

## 3. 做一些小修改，方便我們之後做財經資料研究

In [ ]:
from finlab.crawler import commit

commit()

## 4. 將做完修改的資料拿出來

In [1]:
df = pd.read_pickle("history/items/price/收盤價.pkl")
df.head()

NameError: name 'pd' is not defined

# AUTO update 每日自動執行更新

有時候你可能會希望將所有的程式

每天自動執行就好，不然還要開jupyter -> 啟動notebook -> 執行GUI -> 點擊好幾次GUI

才能完成更新的動作
.
接下來我們就來教怎麼樣直接一次更新吧！

In [1]:
from finlab.crawler import table_date_range, update_table, to_pickle, commit
import datetime
from inspect import signature
from finlab.crawler import (

    # 爬蟲們
    crawl_price,
    crawl_bargin,
    crawl_pe,
    crawl_monthly_report,
    crawl_finance_statement_by_date,
    crawl_twse_divide_ratio,
    crawl_otc_divide_ratio,
    crawl_twse_cap_reduction,
    crawl_otc_cap_reduction,
    crawl_benchmark,
    
    # 時間的range
    date_range,
    month_range,
    season_range,
    
    # GUI介面
    widget, out,
)

def auto_update(table_name, crawl_function, time_range=None):

    sig = signature(crawl_function)
    print(table_name)

    if len(sig.parameters) != 0:
        first_date, last_date = table_date_range(table_name)
        dates = time_range(last_date, datetime.datetime.now())
        if dates:
            update_table(table_name, crawl_function, dates)
    else:
        df = crawl_function()
        to_pickle(df, table_name)
        

auto_update('price', crawl_price, date_range)
auto_update('bargin_report', crawl_bargin, date_range)
auto_update('pe', crawl_pe, date_range)
auto_update('monthly_report', crawl_monthly_report, month_range)
auto_update('twse_divide_ratio', crawl_twse_divide_ratio)
auto_update('otc_divide_ratio', crawl_otc_divide_ratio)
auto_update('twse_cap_reduction', crawl_twse_cap_reduction)
auto_update('otc_cap_reduction', crawl_otc_cap_reduction)

commit()

ValueError: numpy.ndarray size changed, may indicate binary incompatibility. Expected 88 from C header, got 80 from PyObject

## 加入新的benchmark data

In [ ]:
# 時間物件
import datetime
import requests
import pandas as pd
from io import StringIO


# 下載某天的資料

date = datetime.date(2018,1,3)

def crawl_benchmark(date):
    date_str = date.strftime('%Y%m%d')
    res = requests.get("http://www.twse.com.tw/exchangeReport/MI_5MINS_INDEX?response=csv&date=" + 
                       date_str + "&_=1544020420045")
    
    if len(res.text) < 100:
        print('holiday')
        return pd.DataFrame()

    # 利用 pandas 將資料整理成表格
    df = pd.read_csv(StringIO(res.text.replace("=","")), header=1, index_col='時間')

    # 資料處理
    df = df.dropna(how='all', axis=0).dropna(how='all', axis=1)
    df.index = pd.to_datetime(date.strftime('%Y %m %d ') + pd.Series(df.index))
    df = df.apply(lambda s: s.astype(str).str.replace(",", "").astype(float))

    df.head()

    df.index.name = 'date'
    df = df.reset_index()
    df['stock_id'] = '台股指數'
    df = df.set_index(['stock_id', 'date'])
    return df

from finlab.crawler import widget, date_range, out

widget('benchmark', crawl_benchmark, date_range)

out.clear_output()
display(out)

## 查看存放好的benchmark data

In [ ]:
from finlab.crawler import commit

commit()

In [ ]:
from finlab.data import Data

data = Data()

df = data.get("發行量加權股價指數")
df.tail()